# use scripts/preprocessing to add columns!

# Imports

In [1]:
# standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic lines
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import re

# Load data

## import get_data() from scripts

In [3]:
#import from my data.py
#https://stackoverflow.com/a/57790804/6163759

import sys,os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    scripts_path = os.path.join(module_path, "scripts")
    if scripts_path not in sys.path:
        sys.path.append(scripts_path)
    display(sys.path)
    
from data import get_data

['/home/louis/code/JammyNinja/Sandbox/Chess_analysis/notebooks',
 '/home/louis/code/lewagon/data/04-Decision-Science/01-Project-Setup/context-and-setup',
 '/home/louis/code/JammyNinja/Sandbox/Chess_analysis/notebooks',
 '/home/louis/.pyenv/versions/3.10.6/lib/python310.zip',
 '/home/louis/.pyenv/versions/3.10.6/lib/python3.10',
 '/home/louis/.pyenv/versions/3.10.6/lib/python3.10/lib-dynload',
 '',
 '/home/louis/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages',
 '/home/louis/code/JammyNinja/Sandbox/Chess_analysis/scripts']

In [4]:
! ls ../data

all_games_2023-01_to_2024-06_all_cols.csv
all_games_2023-01_to_2024-06_raw.csv
all_games_2023-01_to_2024-09_all_new_cols.csv
all_games_2023-01_to_2024-09_raw.csv
all_games_2023-01_to_2024-10_all_new_cols.csv
all_games_2023-01_to_2024-10_raw.csv
all_games_2023-01_to_2024-11_all_new_cols.csv
all_games_2023-01_to_2024-11_raw.csv
all_games_2024-04_to_2024-06_all_cols.csv
all_games_2024-04_to_2024-06_raw.csv
select_games_2023-01_to_2024-06_select_cols.csv
select_games_2023-01_to_2024-09_select_cols.csv
select_games_2023-01_to_2024-10_select_cols.csv
select_games_2023-01_to_2024-11_select_cols.csv
select_games_2024-04_to_2024-06_select_cols.csv


In [5]:
#ye olde waye
# filename = "all_games_df_2023-01_to_2024-05.csv"
# filename = "all_games_2023-01_to_2024-06_all_cols.csv"
# filepath = os.path.join("..","data", filename)
# all_games_df_raw = pd.read_csv(filepath)

In [6]:
all_games_df_raw = get_data(df_name="all_games", descriptor="raw")

Looking for all games of chess played on Chess.com by JammyNinja
          between 2023-01 and 2024-11
file /home/louis/code/JammyNinja/Sandbox/Chess_analysis/data/all_games_2023-01_to_2024-11_raw.csv already exists!
Do you want to download all games again?
>>(Have you played more since last running this?)
y/n:n


In [7]:
type(all_games_df_raw)

pandas.core.frame.DataFrame

In [8]:
from preprocessing import preprocess
test_preproc = preprocess(all_games_df_raw)

Created win/rating columns:
- higher_rated_colour
- winner
- winner_rating_diff
- user_rating_diff
- user_rating
- user_win
- user_colour
Created moves columns:
- move_numbers
- white_moves
- white_clock
- black_moves
- black_clock
- w_cstl_side
- w_cstl_move
- b_cstl_side
- b_cstl_move
Created final position columns:
- winner_ttl_pieces_count
- loser_ttl_pieces_count
- winner_pawns_count
- loser_pawns_count
- winner_pieces_only_count
- loser_pieces_only_count
- pieces_count_diff
- pawns_count_diff
- winner_material_ttl
- loser_material_ttl
- winner_pcs
- loser_pcs
Created castling columns:
- user_castled
- opp_castled

Started with:
- date
- start_time
- end_time
- time_class
- time_control
- rated
- rules
- url
- moves
- final_position_fen
- opening_code
- opening_name
- white_username
- black_username
- result
- white_rating
- black_rating
- white_result
- black_result
file /home/louis/code/JammyNinja/Sandbox/Chess_analysis/data/all_games_2023-01_to_2024-11_all_new_cols.csv already 

In [7]:
X = all_games_df_raw.copy()

In [8]:
all_games_df_raw

,date,start_time,end_time,time_class,time_control,rated,rules,url,moves,final_position_fen,opening_code,opening_name,white_username,black_username,result,white_rating,black_rating,white_result,black_result
0,2023-01-07,2023-01-07 08:17:06,2023-01-07 14:35:53,daily,1/604800,True,chess,https://www.chess.com/game/daily/437377841,1. e4 {[%clk 0:56:50.2] 1... c6 {[%clk 0:27:49...,2r1k2r/pp3ppp/q7/4pb2/1b1P4/1P2B3/1P3PPP/K2R2N...,B10,Caro-Kann-Defense-Hillbilly-Attack-2...d5,sylvainau,JammyNinja,0-1,1298,1461,checkmated,win
1,2023-01-14,2023-01-14 11:34:48,2023-01-14 15:09:28,daily,1/604800,True,chess,https://www.chess.com/game/daily/448245095,1. e4 {[%clk 0:02:24.3] 1... c5 {[%clk 0:15:39...,r2qr3/1b2bpkp/p2p1n2/1p4B1/3Q4/2P2N2/PP3PPP/3R...,B22,Alapin-Sicilian-Defense-2...Nc6-3.d4,JammyNinja,Timkee,0-1,1429,1234,timeout,win
2,2023-01-30,2023-01-30 17:32:36,2023-01-30 07:50:02,daily,1/604800,True,chess,https://www.chess.com/game/daily/436095119,1. e4 {[%clk 0:02:06.5] 1... c6 {[%clk 0:27:48...,rr4k1/3n2pp/2p2p2/p2pp3/P7/1P5P/1RPPNPP1/5RK1 ...,B10,Caro-Kann-Defense-2.Nf3-d5-3.exd5-cxd5,ELNAHAS4,JammyNinja,0-1,1427,1449,timeout,win
3,2023-01-30,2023-01-30 21:29:25,2023-01-30 21:39:52,blitz,300,True,chess,https://www.chess.com/game/live/68853253455,1. Nf3 {[%clk 0:04:54.9] 1... d5 {[%clk 0:04:5...,8/8/4K3/8/8/8/4Q3/6k1 b - -,A09,Reti-Opening-Reti-Gambit-Accepted,ivangard57,JammyNinja,1-0,1022,1030,win,timeout
4,2023-02-07,2023-02-07 20:18:19,2023-02-07 19:25:50,daily,1/604800,True,chess,https://www.chess.com/game/daily/472634081,1. c4 {[%clk 0:18:46.2] 1... e5 {[%clk 4:14:19...,rnbqk1nr/pppp1ppp/8/2b1p3/2P5/2N2P2/PP1PP1PP/R...,A21,English-Opening-Reversed-Sicilian-Variation,ajbouchie,JammyNinja,1/2-1/2,1584,1449,agreed,agreed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4539,2024-11-05,2024-11-05 14:52:33,2024-11-05 14:54:37,bullet,60,True,chess,https://www.chess.com/game/live/124557516553,1. e4 {[%clk 0:01:00] 1... e5 {[%clk 0:01:00] ...,8/pp4p1/2n5/5k2/3p1B1p/P2P1P2/2P2KP1/6N1 w - -,C26,Vienna-Game-Falkbeer-Variation,JammyNinja,Aakula22,0-1,1349,1370,timeout,win
4540,2024-11-05,2024-11-05 14:54:40,2024-11-05 14:56:44,bullet,60,True,chess,https://www.chess.com/game/live/124557542239,1. e4 {[%clk 0:01:00] 1... d5 {[%clk 0:01:00] ...,8/4RN2/7p/8/5R2/7P/P1r2kPK/8 b - -,B01,Scandinavian-Defense-Mieses-Kotrc-Variation......,Aakula22,JammyNinja,1-0,1378,1341,win,checkmated
4541,2024-11-05,2024-11-05 14:56:47,2024-11-05 14:59:00,bullet,60,True,chess,https://www.chess.com/game/live/124557568389,1. e4 {[%clk 0:01:00] 1... e5 {[%clk 0:00:59.9...,6r1/1qkr4/2pp4/8/7P/P3p2K/1P4P1/2R5 b - -,C26,Vienna-Game-Falkbeer-Stanley-Variation-3...Bc5,JammyNinja,Aakula22,1-0,1350,1369,win,timeout
4542,2024-11-05,2024-11-05 14:59:05,2024-11-05 14:59:42,bullet,60,True,chess,https://www.chess.com/game/live/124557596139,1. e4 {[%clk 0:01:00] 1... d5 {[%clk 0:01:00] ...,r3k2r/pp1nQpb1/2p2pbp/8/3P4/3B1NN1/qPP2PPP/2KR...,D00,Queens-Pawn-Opening-Blackmar-Diemer-von-Popiel...,Aakula22,JammyNinja,1-0,1377,1342,win,checkmated


In [9]:
test_game = all_games_df_raw.query("time_class == 'bullet'").iloc[1]
# test_moves = test_game["moves"]
test_game

date                                                         2023-09-13
start_time                                          2023-09-13 19:47:02
end_time                                            2023-09-13 19:48:51
time_class                                                       bullet
time_control                                                       60+1
rated                                                              True
rules                                                             chess
url                         https://www.chess.com/game/live/88373428625
moves                 1. e4 {[%clk 0:01:01] 1... e5 {[%clk 0:01:01] ...
final_position_fen    2kr3r/1pp1q1pp/p1n2n2/2b1p3/4p3/1BN5/PPPPQPPP/...
opening_code                                                        C47
opening_name                        Four-Knights-Game-Italian-Variation
white_username                                               JammyNinja
black_username                                                in

# Add new columns

In [10]:
def process_moves(row):
    """
        Looks at 'moves' column which is pgn string, extracts each players moves and when they castled
        ['move_numbers', 'white_moves', 'white_clock', 'black_moves', 'black_clock', 
        'w_cstl_side', 'w_cstl_move', 'b_cstl_side', 'b_cstl_move']
    """
    #.apply() complained that it was float when didn't manually convert to string?
    moves = str(row["moves"])
    moves = moves.replace("{","")

    move_nums = moves.split()[::8]
    white_moves = moves.split()[1::8]
    black_moves = moves.split()[5::8]      
    
    white_clock = [m[:-1] for m in moves.split()[3::8] ]
    black_clock = [m[:-1] for m in moves.split()[7::8] ]
     
    if len(black_moves) < len(white_moves):
        black_moves.append(np.nan)
        black_clock.append(np.nan)
    
    #deal with castling
    w_castled, w_castled_side = 0,np.nan
    b_castled, b_castled_side = 0, np.nan
    
    for move_num, w_move, b_move in zip(move_nums, white_moves, black_moves):
        move_num = int(move_num[:-1]) #7. -> 7
        
        if w_move == "O-O":
            w_castled_side = "King"
            w_castled = move_num
        elif w_move == "O-O-O":
            w_castled_side = "Queen"
            w_castled = move_num
            
        if b_move == "O-O":
            b_castled_side = "King"
            b_castled = move_num
        elif b_move == "O-O-O":
            b_castled_side = "Queen"
            b_castled = move_num
            
#     print(f"white castled {w_castled_side} side on move {w_castled}")
#     print(f"black castled {b_castled_side} side on move {b_castled}")
    
    df_dict = {
        "move_numbers" : move_nums,
        "white_moves"  : white_moves,
        "white_clock" : white_clock,
        "black_moves"  : black_moves,
        "black_clock" : black_clock,
        
        "w_cstl_side" : w_castled_side,
        "w_cstl_move" : w_castled,        
        "b_cstl_side" : b_castled_side,
        "b_cstl_move" : b_castled
    }
    
    return df_dict
    
process_moves(test_game)

{'move_numbers': ['1.',
  '2.',
  '3.',
  '4.',
  '5.',
  '6.',
  '7.',
  '8.',
  '9.',
  '10.',
  '11.',
  '12.'],
 'white_moves': ['e4',
  'Nf3',
  'Nc3',
  'Bc4',
  'Bb3',
  'Ng5',
  'Nxe6',
  'Bxe6',
  'Qe2',
  'Bb3',
  'Qb5',
  'Qe2'],
 'white_clock': ['0:01:01',
  '0:01:00.2',
  '0:00:57.9',
  '0:00:57.3',
  '0:00:56.3',
  '0:00:49.1',
  '0:00:48.5',
  '0:00:48.5',
  '0:00:46.3',
  '0:00:43.9',
  '0:00:43',
  '0:00:40.7'],
 'black_moves': ['e5',
  'Nf6',
  'Nc6',
  'd5',
  'dxe4',
  'Be6',
  'fxe6',
  'Bc5',
  'Qe7',
  'O-O-O',
  'a6',
  nan],
 'black_clock': ['0:01:01',
  '0:01:00.3',
  '0:00:58.3',
  '0:00:50.9',
  '0:00:47.5',
  '0:00:34.6',
  '0:00:33.6',
  '0:00:30.1',
  '0:00:26.2',
  '0:00:11.1',
  '0:00:05.5',
  nan],
 'w_cstl_side': nan,
 'w_cstl_move': 0,
 'b_cstl_side': 'Queen',
 'b_cstl_move': 10}

## pipeline test

In [11]:
# Pipelines
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion, make_union
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_regression, make_classification, make_multilabel_classification, make_blobs
import pickle

In [45]:
def process_moves_pipe(df):
    """
        Looks at 'moves' column which is pgn string, extracts each players moves and when they castled
        ['move_numbers', 'white_moves', 'white_clock', 'black_moves', 'black_clock', 
        'w_cstl_side', 'w_cstl_move', 'b_cstl_side', 'b_cstl_move']
    """
    def process_moves(row):
        #.apply() complained that it was float when didn't manually convert to string?
        moves = str(row["moves"])
        moves = moves.replace("{","")

        move_nums = moves.split()[::8]
        white_moves = moves.split()[1::8]
        black_moves = moves.split()[5::8]      

        white_clock = [m[:-1] for m in moves.split()[3::8] ]
        black_clock = [m[:-1] for m in moves.split()[7::8] ]

        if len(black_moves) < len(white_moves):
            black_moves.append(np.nan)
            black_clock.append(np.nan)

        #deal with castling
        w_castled, w_castled_side = 0,np.nan
        b_castled, b_castled_side = 0, np.nan

        for move_num, w_move, b_move in zip(move_nums, white_moves, black_moves):
            move_num = int(move_num[:-1]) #7. -> 7

            if w_move == "O-O":
                w_castled_side = "King"
                w_castled = move_num
            elif w_move == "O-O-O":
                w_castled_side = "Queen"
                w_castled = move_num

            if b_move == "O-O":
                b_castled_side = "King"
                b_castled = move_num
            elif b_move == "O-O-O":
                b_castled_side = "Queen"
                b_castled = move_num

        df_dict = {
            "move_numbers" : move_nums,
            "white_moves"  : white_moves,
            "white_clock" : white_clock,
            "black_moves"  : black_moves,
            "black_clock" : black_clock,

            "w_cstl_side" : w_castled_side,
            "w_cstl_move" : w_castled,        
            "b_cstl_side" : b_castled_side,
            "b_cstl_move" : b_castled
        }
        return pd.Series(df_dict)

    out_df = df.apply(process_moves, axis=1, result_type="reduce")
    return out_df 

# process_moves_pipe(test_game)
process_moves_pipe(all_games_df_raw.iloc[:2,:])
# pd.DataFrame.apply()

,move_numbers,white_moves,white_clock,black_moves,black_clock,w_cstl_side,w_cstl_move,b_cstl_side,b_cstl_move
0,"[1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11.,...","[e4, Bc4, exd5, Bb3, d4, Qe2, Qd3, Bg5, Qe3, N...","[0:56:50.2, 0:07:35.1, 0:55:52.1, 0:51:51.9, 1...","[c6, d5, cxd5, Nf6, Nc6, Nxd4, e5, Be6, Qa5+, ...","[0:27:49.5, 3:13:05.6, 9:10:15.2, 1:04:14.9, 6...",NaN,0,NaN,0
1,"[1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11.,...","[e4, c3, d4, dxc5, cxd6, Bb5, Bf4, Nd2, Bd3, N...","[0:02:24.3, 0:02:01.1, 1:40:11, 12:21:05.2, 6:...","[c5, Nc6, d6, b6, exd6, Bb7, Nf6, a6, Be7, O-O...","[0:15:39.3, 0:19:39.7, 0:16:34.7, 0:13:40.6, 0...",King,11,King,10


In [47]:
moves_proc = FunctionTransformer(process_moves_pipe)

moves_pipe = Pipeline([
    ("process_moves", moves_proc)
])
moves_pipe.fit_transform(X.iloc[:5])

,move_numbers,white_moves,white_clock,black_moves,black_clock,w_cstl_side,w_cstl_move,b_cstl_side,b_cstl_move
0,"[1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11.,...","[e4, Bc4, exd5, Bb3, d4, Qe2, Qd3, Bg5, Qe3, N...","[0:56:50.2, 0:07:35.1, 0:55:52.1, 0:51:51.9, 1...","[c6, d5, cxd5, Nf6, Nc6, Nxd4, e5, Be6, Qa5+, ...","[0:27:49.5, 3:13:05.6, 9:10:15.2, 1:04:14.9, 6...",NaN,0,NaN,0
1,"[1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11.,...","[e4, c3, d4, dxc5, cxd6, Bb5, Bf4, Nd2, Bd3, N...","[0:02:24.3, 0:02:01.1, 1:40:11, 12:21:05.2, 6:...","[c5, Nc6, d6, b6, exd6, Bb7, Nf6, a6, Be7, O-O...","[0:15:39.3, 0:19:39.7, 0:16:34.7, 0:13:40.6, 0...",King,11,King,10
2,"[1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11.,...","[e4, Nf3, exd5, b3, h3, Qxf3, Bb5+, O-O, Bb2, ...","[0:02:06.5, 0:15:03.3, 0:00:03.4, 0:12:53.2, 0...","[c6, d5, cxd5, Bg4, Bxf3, e6, Nc6, Bc5, Nf6, O...","[0:27:48.3, 2:08:23.9, 1:21:24.6, 5:33:48.6, 1...",King,8,King,10
3,"[1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11.,...","[Nf3, c4, e4, a4, axb5, Nc3, b3, Bxa6, Rxa6, Q...","[0:04:54.9, 0:04:53.1, 0:04:50.9, 0:04:48.6, 0...","[d5, dxc4, b5, Bd7, Bxb5, Ba6, cxb3, Nxa6, Qd3...","[0:04:58, 0:04:56, 0:04:47.6, 0:04:42.5, 0:04:...",King,16,Queen,12
4,"[1., 2., 3.]","[c4, Nc3, f3]","[0:18:46.2, 0:42:16.7, 0:06:49.3]","[e5, Bc5, nan]","[4:14:19.7, 11:29:36.4, nan]",NaN,0,NaN,0


In [55]:
moves_pipe.get_feature_names_out()

AttributeError: Estimator process_moves does not provide get_feature_names_out. Did you mean to call pipeline[:-1].get_feature_names_out()?

In [54]:
moves_pipe.fit_transform(X.iloc[4:5])
X.iloc[4]

date                                                         2023-02-07
start_time                                          2023-02-07 20:18:19
end_time                                            2023-02-07 19:25:50
time_class                                                        daily
time_control                                                   1/604800
rated                                                              True
rules                                                             chess
url                          https://www.chess.com/game/daily/472634081
moves                 1. c4 {[%clk 0:18:46.2] 1... e5 {[%clk 4:14:19...
final_position_fen    rnbqk1nr/pppp1ppp/8/2b1p3/2P5/2N2P2/PP1PP1PP/R...
opening_code                                                        A21
opening_name                English-Opening-Reversed-Sicilian-Variation
white_username                                                ajbouchie
black_username                                               Jam

In [10]:
def process_winner_rating(row):
    
    winner = "white" if row["result"] == "1-0" else "black" if row["result"] == "0-1" else "DRAW"
    
    w_rating, b_rating = row["white_rating"], row["black_rating"] #reduce line size
    
    higher_rated = "white" if w_rating > b_rating else "black" #assumes draw is good for black
    
    user_colour = "white" if row["white_username"] == "JammyNinja" else "black"
        
    def did_user_win(user = "JammyNinja"):
        if winner == "white" and row["white_username"] == user:
            return True
        if winner == "black" and row["black_username"] == user:
            return True
        if winner == "draw":
            return "DRAW"
        return False
    
    user_win = did_user_win()
    
    user_rating = w_rating if user_colour == "white" else b_rating
    
    winner_rating_diff = w_rating - b_rating if winner == "white" else b_rating - w_rating
    user_rating_diff = w_rating - b_rating if user_colour == "white" else b_rating - w_rating
        
    return {
        "higher_rated_colour" : higher_rated,
        "winner"       : winner,
        "winner_rating_diff"  : winner_rating_diff,
        "user_rating_diff" : user_rating_diff,
        "user_rating" : user_rating,
        "user_win"     : user_win,
        "user_colour" : user_colour
    }
process_winner_rating(all_games_df_raw.iloc[4]).keys()

dict_keys(['higher_rated_colour', 'winner', 'winner_rating_diff', 'user_rating_diff', 'user_rating', 'user_win', 'user_colour'])

In [11]:
def user_castled(row):
#         print(row)
        user_castled = False
        opp_castled = False

        w_cstl_move = row["w_cstl_move"]
        b_cstl_move = row["b_cstl_move"]

        if row["user_colour"] == "white":
            if w_cstl_move > 0:
                user_castled = True
                user_cstl_diff = w_cstl_move - b_cstl_move  
            if b_cstl_move > 0:
                opp_castled = True
        else: #user is black
            if b_cstl_move > 0:
                user_castled = True
                user_cstl_diff = b_cstl_move - w_cstl_move 
            if w_cstl_move > 0:
                opp_castled = True
                
        
        #create columns to do with who castled first         
        #null if user didn't castle
        if not user_castled:
            user_cstl_diff = np.nan
        #WHAT IF I CASTLE AND THEY DONT - cant subtract?
        
        return {
            "user_castled" : user_castled,
            "opp_castled" : opp_castled,
#             "user_cstl_diff" : user_cstl_diff,
#             "winner_cstl_diff" : win_cstl_diff
        }
# user_castled(all_games_df.iloc[80])
# display(all_games_df.iloc[80])

In [12]:
test_game

date                                                         2023-09-13
start_time                                          2023-09-13 19:47:02
end_time                                            2023-09-13 19:48:51
time_class                                                       bullet
time_control                                                       60+1
rated                                                              True
rules                                                             chess
url                   https://www.chess.com/game/47ff5565-526e-11ee-...
moves                 1. e4 {[%clk 0:01:01] 1... e5 {[%clk 0:01:01] ...
final_position_fen    2kr3r/1pp1q1pp/p1n2n2/2b1p3/4p3/1BN5/PPPPQPPP/...
opening_code                                                        C47
opening_name                        Four-Knights-Game-Italian-Variation
white_username                                               JammyNinja
black_username                                                in

In [13]:
def pieces_final_position(game):
    """
    starts with eighth rank, first file to eighth
    white is uppercase, black lowercase
    p: pawn, r: rook, n: knight, b: bishop, q : queen, k : king 
    """
    fen = game["final_position_fen"]
    fin_pos = fen.split()[0]
    
    board = fin_pos.split("/")
    ttl_white_pcs, ttl_black_pcs = 0,0
    
    #white pieces are uppercase, black lowercase
    w_pcs = [piece for piece in "".join(board) if piece.isupper()]
    b_pcs = [piece for piece in "".join(board) if piece.islower()]
    #count total pieces (including king), and also count just the pawns
    ttl_white_pcs = len(w_pcs)
    ttl_black_pcs = len(b_pcs)
    ttl_white_pwns = len([p for p in w_pcs if p == "P"])
    ttl_black_pwns = len([p for p in b_pcs if p == "p"])

    #create material counts
    def sum_piece_values(pcs):
        """
        Accepts as input b_pcs or w_bcs and outputs the total value of what's given.
        """
        # dictionary for piece values
        chess_piece_values = {
            'P': 1,  # Pawn
            'N': 3,  # Knight
            'B': 3,  # Bishop
            'R': 5,  # Rook
            'Q': 9,  # Queen
            'K': 0,   # King (often considered to have infinite value, but in practical terms, it is 0 for valuation purposes)
        }
        count = 0
        for i in pcs:
            count += chess_piece_values[i.upper()]
        return count
    
    w_material_count=sum_piece_values(w_pcs)
    b_material_count=sum_piece_values(b_pcs)
        
    if game.winner == "white":
        win_pcs, lose_pcs = w_pcs, b_pcs
        winner_ttl_pieces_count = ttl_white_pcs
        loser_ttl_pieces_count = ttl_black_pcs
        winner_pawns_count = ttl_white_pwns
        loser_pawns_count = ttl_black_pwns
        winner_material_ttl = w_material_count
        loser_material_ttl = b_material_count        
        
    else: #includes DRAWs
        win_pcs, lose_pcs = b_pcs, w_pcs
        winner_ttl_pieces_count = ttl_black_pcs
        loser_ttl_pieces_count = ttl_white_pcs
        winner_pawns_count = ttl_black_pwns
        loser_pawns_count = ttl_white_pwns
        winner_material_ttl = b_material_count
        loser_material_ttl = w_material_count
    
    #count of just the major/minor pieces, -1 to not include King,    
    winner_pieces_only_count = winner_ttl_pieces_count - winner_pawns_count -1
    loser_pieces_only_count = loser_ttl_pieces_count - loser_pawns_count -1
    
    output = {
        #pieces on the board including king
        "winner_ttl_pieces_count" : winner_ttl_pieces_count, 
        "loser_ttl_pieces_count" : loser_ttl_pieces_count,
#       #count pawns
        "winner_pawns_count" : winner_pawns_count, 
        "loser_pawns_count" : loser_pawns_count,
        #count of just the major/minor pieces
        "winner_pieces_only_count" : winner_pieces_only_count,
        "loser_pieces_only_count" : loser_pieces_only_count,
        "pieces_count_diff" : winner_pieces_only_count - loser_pieces_only_count,
        "pawns_count_diff" : winner_pawns_count - loser_pawns_count,
        #count the material worth
        "winner_material_ttl" : winner_material_ttl,
        "loser_material_ttl" : loser_material_ttl,
        #might as well throw the pieces in too
        "winner_pcs" : win_pcs,
        "loser_pcs"  : lose_pcs,
    }
    
    return output

test_game = all_games_df_raw.query("time_class == 'bullet'").iloc[9]
# print(test_game.final_position_fen)
pieces_final_position(test_game).keys()

AttributeError: 'Series' object has no attribute 'winner'

In [ ]:
def add_new_columns(games_df_in, printing=True):
    
    orig_cols = games_df_in.columns
    
    #uses winner_rating function to calculate the winner and rating swing
    win_rating_cols = games_df_in.apply(process_winner_rating, axis="columns", result_type="expand")
    
    #uses process moves function to break down the moves into w/b, and when castled
    move_cols =       games_df_in.apply(process_moves, axis="columns", result_type="expand")
    
    #add the above columns for now, so that the below funcs know who the winner was
    games_df_win = pd.concat([win_rating_cols, move_cols, games_df_in], axis=1)

    #drop duplicated columns
    #https://stackoverflow.com/questions/32041245/fast-method-for-removing-duplicate-columns-in-pandas-dataframe
    unique_cols = ~games_df_win.columns.duplicated()
    games_df_win = games_df_win.loc[:, unique_cols]
    
    display(games_df_win.columns)
    #create final position columns, gets the material left on the board
    final_pos_cols = games_df_win.apply(pieces_final_position, axis="columns", result_type="expand")
    
    #now calculate castle cols using some of the above columns
    cstl_cols =       games_df_win.apply(user_castled, axis="columns", result_type="expand")
    games_df_out = pd.concat([games_df_win, cstl_cols, final_pos_cols], axis=1)
    
    if printing:
        print("Created win/rating columns:\n" + "\n".join([f"- {col}" for col in win_rating_cols]))
        print("Created moves columns:\n" + "\n".join([f"- {col}" for col in move_cols]))
        print("Created final position columns:\n" + "\n".join([f"- {col}" for col in final_pos_cols]))
        print("Created castling columns:\n" + "\n".join([f"- {col}" for col in cstl_cols]))
        print("\nStarted with:\n" + "\n".join([f"- {col}" for col in orig_cols]))
    return games_df_out
    
all_games_df = add_new_columns(all_games_df_raw)
all_games_df.sample(2)

In [ ]:
all_games_df.columns

In [ ]:
test_game = all_games_df.query("time_class == 'bullet'").iloc[10]
test_game

## order the columns

In [ ]:
all_cols_order = [
    #meta
    'date', 'start_time', 'end_time',
    'time_class', 'time_control', 'rated', 'rules',
    'url', 'moves',
    'opening_code', 'opening_name',
    'white_username', 'black_username',
    'white_clock', 'black_clock',
    
    #ratings/results
    'white_rating', 'black_rating',
    'result', 'white_result', 'black_result',
    'higher_rated_colour', 'winner', 
    'winner_rating_diff', 'user_rating_diff',
    'user_win', 'user_colour', 
    'user_rating', 'move_numbers',
     
    #moves
    'white_moves',  'black_moves',
    'w_cstl_side', 'w_cstl_move', 
    'b_cstl_side', 'b_cstl_move',
    'user_castled', 'opp_castled',
    
    #final position
     'final_position_fen',
    'winner_ttl_pieces_count','loser_ttl_pieces_count',
    'winner_pawns_count', 'loser_pawns_count',
    'winner_pieces_only_count', 'loser_pieces_only_count',
    'pieces_count_diff', 'pawns_count_diff',
    'winner_material_ttl', 'loser_material_ttl',
    'winner_pcs', 'loser_pcs'
]

cols_pre = all_games_df.columns
all_games_df = all_games_df[all_cols_order]

removed_cols = [col for col in all_games_df.columns if col not in cols_pre]
if removed_cols:
    print("You have excluded the following columns:\n" + "\n".join([f"- {col}" for col in removed_cols]))

## Save to file

In [ ]:
# filename = "all_games_all_cols.csv"
# all_games_df.to_csv(os.path.join("..","data",filename), index=False)

# Selecting data

In [ ]:
def exclude_games(df, diff_max = 400):
    #if True, remove corresponding games
    constraints = {
        "chess960" : True,
        "daily" : True,
        "unrated" : True,
        "rating": True,
        "drawn" : False
    }
    
    queries = {
        #"title" : [ False_query, True_query] -> [to remove, to keep]
        "chess960" : ["rules != 'chess'", "rules == 'chess'"],
        "unrated" : ["rated != True", "rated == True"],
        "daily" : ["time_class == 'daily'", "time_class != 'daily'"],
        "rating" : [f"abs(winner_rating_diff) > {diff_max}", f"abs(winner_rating_diff) <= {diff_max}"],
        "drawn" :["winner == 'DRAW'", "winner != 'DRAW'"]
    }
    
    selected_games_df = df.copy()
    
    for constraint, exclude in constraints.items():
        exclude_df = df.query(queries[constraint][0])            
        include_df = selected_games_df.query(queries[constraint][1])
        suffix = "include"
        
        if exclude:
            selected_games_df = include_df
            suffix="exclude"
            
        if constraint != "rating":
            print(f"Found {len(exclude_df)} {constraint} games to {suffix}")
        else:
            print(f"Found {len(exclude_df)} games with a rating difference over {diff_max} to {suffix}")

    print(f"\nExcluded a total of {len(df) - len(selected_games_df)} rows due to constraints")
    print(f"Leaving {len(selected_games_df)} games to work with")
    
    return selected_games_df
    
exclude_games(all_games_df)
#test not excluding, test rating change

### Exclude rows

In [ ]:
selected_games_df = all_games_df.copy()

#exclude any chess960 games
selected_games_df = all_games_df.query("rules == 'chess'")

#exclude unrated games
selected_games_df = selected_games_df.query("rated == True")

#exclude daily games
selected_games_df = selected_games_df.query("time_class != 'daily'")

#exclude excess rating diff
diff_max = 400
selected_games_df = selected_games_df.query(f"abs(winner_rating_diff) <= {diff_max}")

print(f"Excluded {len(all_games_df) - len(selected_games_df)} rows due to constraints")
print(f"Leaves {len(selected_games_df)} games to work with")

draws_df = selected_games_df.query("winner=='DRAW'") #result=='1/2-1/2'
print(f"Of which {len(draws_df)} are draws")

selected_games_df.sample(3)

### Select columns

In [ ]:
select_cols = [
        #metadata
        'date', 'url',
        'start_time', 'end_time', 
        'time_class', 'time_control',
#         'rated', 'rules', #used to filter out unwanted games
        'white_username', 'black_username', "user_colour",
        #ratings/result
        'result', 'winner', 'higher_rated_colour',
        'white_rating', 'black_rating', 
        'white_result', 'black_result',
        'winner_rating_diff', "user_rating_diff", "user_rating",
        #specific moves of the game
        'opening_name', 'opening_code',
        'w_cstl_side', 'w_cstl_move', 
        'b_cstl_side', 'b_cstl_move',
        'user_castled', 'opp_castled',
        #final position stats
        'winner_ttl_pieces_count', 'loser_ttl_pieces_count',
        'winner_pawns_count', 'loser_pawns_count',
        'winner_pieces_only_count', 'loser_pieces_only_count',
        'pieces_count_diff', 'pawns_count_diff', 
        'winner_material_ttl', 'loser_material_ttl',
        'winner_pcs', 'loser_pcs',
        #target
        'user_win' 
]

selected_games_df = selected_games_df[select_cols]

removed_cols = [col for col in all_games_df.columns if col not in selected_games_df.columns]
print("You have excluded the following columns:\n" + "\n".join([f"- {col}" for col in removed_cols]))



## Inspect selected data

In [ ]:
selected_games_df.sample(5)

## Save selected data

In [ ]:
dates = [part for part in filename.split('_') if '-' in part]
print(dates) 

In [ ]:
filename_select = f"select_games_{dates[0]}_to_{dates[1]}_select_cols.csv"
selected_games_df.to_csv(os.path.join("..","data",filename), index=False)